# Import Modules

In [4]:
import warnings
warnings.filterwarnings('ignore')

from src import detect_faces, show_bboxes
from PIL import Image

import torch
from torchvision import transforms, datasets
import numpy as np
import os

# Path Definition

In [5]:
dataset_path = '../Dataset/emotiw/'
face_coordinates_directory = '../Dataset/FaceCoordinates/'
processed_dataset_path = '../Dataset/CroppedFaces/'

# Load Train and Val Dataset

In [6]:
image_datasets = {x : datasets.ImageFolder(os.path.join(dataset_path, x))
                    for x in ['train', 'val']}

class_names = image_datasets['train'].classes

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: '../Dataset/emotiw/train'

In [ ]:
class_names

In [ ]:
training_dataset = image_datasets['train']
validation_dataset = image_datasets['val']

In [ ]:
neg_train = sorted(os.listdir(dataset_path + 'train/Negative/'))
neu_train = sorted(os.listdir(dataset_path + 'train/Neutral/'))
pos_train = sorted(os.listdir(dataset_path + 'train/Positive/'))

neg_val = sorted(os.listdir(dataset_path + 'val/Negative/'))
neu_val = sorted(os.listdir(dataset_path + 'val/Neutral/'))
pos_val = sorted(os.listdir(dataset_path + 'val/Positive/'))

In [ ]:
neg_train_filelist = [x.split('.')[0] for x in neg_train]
neu_train_filelist = [x.split('.')[0] for x in neu_train]
pos_train_filelist = [x.split('.')[0] for x in pos_train]

neg_val_filelist = [x.split('.')[0] for x in neg_val]
neu_val_filelist = [x.split('.')[0] for x in neu_val]
pos_val_filelist = [x.split('.')[0] for x in pos_val]

In [ ]:
print(neg_train_filelist[:10])
print(neu_train_filelist[:10])
print(pos_train_filelist[:10])

print(neg_val_filelist[:10])
print(neu_val_filelist[:10])
print(pos_val_filelist[:10])

In [ ]:
train_filelist = neg_train_filelist + neu_train_filelist + pos_train_filelist
val_filelist = neg_val_filelist + neu_val_filelist + pos_val_filelist

In [ ]:
print(len(training_dataset))
print(len(validation_dataset))

# Crop Faces

In [ ]:
for i in range(len(training_dataset)):
    try:
        image, label = training_dataset[i]
        face_list = []
        landmarks_new_coordinates = []
        if label == 0:
            if os.path.isfile(processed_dataset_path + 'train/Negative/' + train_filelist[i] + '.npz'):
                print(train_filelist[i] + ' Already present')
                continue

            bbox_lm = np.load(face_coordinates_directory + 'train/Negative/' + train_filelist[i] +'.npz')
            bounding_boxes = bbox_lm['a']
            if bounding_boxes.size == 0 or (bounding_boxes[0] == 0).all():
                print("No bounding boxes for " + train_filelist[i] + ". Adding empty file for the same")
                np.savez(processed_dataset_path + 'train/Negative/' + train_filelist[i], a = np.zeros(1), b = np.zeros(1))
                continue
            landmarks = bbox_lm['b']

            for j in range(len(bounding_boxes)):
                bbox_coordinates = bounding_boxes[j]
                landmark = landmarks[j]
                img_face = image.crop((bbox_coordinates[0], bbox_coordinates[1], bbox_coordinates[2], bbox_coordinates[3]))

                x = bbox_coordinates[0]
                y = bbox_coordinates[1]
                
                for k in range(5):
                    landmark[k] -= x
                    landmark[k+5] -= y
                img_face = np.array(img_face)
                landmark = np.array(landmark)


                if len(face_list) != 0:
                    if img_face.shape[0] == face_list[-1].shape[0]:
                        img_face = image.crop((bbox_coordinates[0] - 1, bbox_coordinates[1] - 1, bbox_coordinates[2], bbox_coordinates[3]))
                        img_face = np.array(img_face)
                        landmark +=1

                face_list.append(img_face)
                landmarks_new_coordinates.append(landmark)
            face_list = np.asarray(face_list)
            landmarks_new_coordinates = np.asarray(landmarks_new_coordinates)
            np.savez(processed_dataset_path + 'train/Negative/' + train_filelist[i], a = face_list, b = landmarks_new_coordinates)

        elif label == 1:

            if os.path.isfile(processed_dataset_path + 'train/Neutral/' + train_filelist[i] + '.npz'):
                print(train_filelist[i] + ' Already present')
                continue

            bbox_lm = np.load(face_coordinates_directory + 'train/Neutral/' + train_filelist[i] +'.npz')
            bounding_boxes = bbox_lm['a']
            if bounding_boxes.size == 0 or (bounding_boxes[0] == 0).all():
                print("No bounding boxes for " + train_filelist[i] + ". Adding empty file for the same")
                np.savez(processed_dataset_path + 'train/Neutral/' + train_filelist[i], a = np.zeros(1), b = np.zeros(1))
                continue
            landmarks = bbox_lm['b']

            for j in range(len(bounding_boxes)):
                bbox_coordinates = bounding_boxes[j]
                landmark = landmarks[j]
                img_face = image.crop((bbox_coordinates[0], bbox_coordinates[1], bbox_coordinates[2], bbox_coordinates[3]))
   
                x = bbox_coordinates[0]
                y = bbox_coordinates[1]
        
                for k in range(5):
                    landmark[k] -= x
                    landmark[k+5] -= y
                img_face = np.array(img_face)
                landmark = np.array(landmark)

                if len(face_list) != 0:
                    if img_face.shape[0] == face_list[-1].shape[0]:
                        img_face = image.crop((bbox_coordinates[0] - 1, bbox_coordinates[1] - 1, bbox_coordinates[2], bbox_coordinates[3]))
                        img_face = np.array(img_face)
                        landmark += 1

                face_list.append(img_face)
                landmarks_new_coordinates.append(landmark)
            face_list = np.asarray(face_list)
            landmarks_new_coordinates = np.asarray(landmarks_new_coordinates)
            np.savez(processed_dataset_path + 'train/Neutral/' + train_filelist[i], a = face_list, b = landmarks_new_coordinates)

        else:
            if os.path.isfile(processed_dataset_path + 'train/Positive/' + train_filelist[i] + '.npz'):
                print(train_filelist[i] + ' Already present')
                continue

            bbox_lm = np.load(face_coordinates_directory + 'train/Positive/' + train_filelist[i] +'.npz')
            bounding_boxes = bbox_lm['a']
            if bounding_boxes.size == 0 or (bounding_boxes[0] == 0).all():
                print("No bounding boxes for " + train_filelist[i] + ". Adding empty file for the same")
                np.savez(processed_dataset_path + 'train/Positive/' + train_filelist[i], a = np.zeros(1), b = np.zeros(1))
                continue
            landmarks = bbox_lm['b']

            for j in range(len(bounding_boxes)):
                bbox_coordinates = bounding_boxes[j]
                landmark = landmarks[j]
                img_face = image.crop((bbox_coordinates[0], bbox_coordinates[1], bbox_coordinates[2], bbox_coordinates[3]))
    
                x = bbox_coordinates[0]
                y = bbox_coordinates[1]
            
                for k in range(5):
                    landmark[k] -= x
                    landmark[k+5] -= y
                img_face = np.array(img_face)
                landmark = np.array(landmark)

                if len(face_list) != 0:
                    if img_face.shape[0] == face_list[-1].shape[0]:
                        img_face = image.crop((bbox_coordinates[0] - 1, bbox_coordinates[1] - 1, bbox_coordinates[2], bbox_coordinates[3]))
                        img_face = np.array(img_face)
                        landmark += 1

                face_list.append(img_face)
                landmarks_new_coordinates.append(landmark)
            face_list = np.asarray(face_list)
            landmarks_new_coordinates = np.asarray(landmarks_new_coordinates)
            np.savez(processed_dataset_path + 'train/Positive/' + train_filelist[i], a = face_list, b = landmarks_new_coordinates)

        if i % 100 == 0:
            print(i)
    except:
        print("Error/interrput at validation dataset file " + val_filelist[i])
        print(bounding_boxes)
        print(landmarks)
        print(bounding_boxes.shape)
        print(landmarks.shape)
        break